In [2]:
import pandas as pd
import duckdb
import altair as alt

#### abril

In [3]:
# ¿Por que razones el voto no sumo a ningun partido?

no_positivos = duckdb.sql("""
    SELECT votos_tipo, SUM(votos_cantidad) AS votos 
    FROM read_parquet('ResultadosElectorales_*.parquet') 
    WHERE cargo_id = 1 AND votos_tipo != 'POSITIVO'
    GROUP BY 1
""").df()
no_positivos

chart = alt.Chart(no_positivos).mark_bar().encode(
    x=alt.X('votos_tipo:N', sort='-y',title=None),
    y='votos:Q'
).properties(
    title='Votos por Tipo',
    width=400
)

chart

alt.Chart(...)

In [4]:
# ¿Cuantos votos recibieron en las elecciones GENERALES los 5 patidos más relevantes?

agrupacion_1v = duckdb.sql("""
  SELECT agrupacion_nombre, sum(votos_cantidad) as votos
  FROM read_parquet('ResultadosElectorales_1v.parquet') 
  WHERE agrupacion_nombre IS NOT NULL AND cargo_id = 1
  GROUP BY agrupacion_nombre
  ORDER BY 2 desc
""").df()

agrupacion_1v['agrupacion_nombre'] = agrupacion_1v['agrupacion_nombre'].replace(
    {'FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD': 'FRENTE DE IZQUIERDA'})
# Formatear los votos
agrupacion_1v['votos_formateados'] = agrupacion_1v['votos'].apply(lambda x: '{:,}'.format(x))
agrupacion_1v

base = alt.Chart(agrupacion_1v).encode(
    x='votos:Q',
    y=alt.Y('agrupacion_nombre:N', sort='-x',title=None),  # Configurar axis=None para ocultar el eje y
    text='votos_formateados'
)

(base.mark_bar() + base.mark_text(align='left', dx=2)).properties(
    title="Voto a Presidente, GENERALES 2023")

alt.LayerChart(...)